In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import h5py

In [4]:
#Fonction :
def initialisation(dimensions):
    
    param = {}
    for l in range(1, len(dimensions)):
        param['W' + str(l)] = np.random.randn(dimensions[l], dimensions[l - 1])
        param['b' + str(l)] = np.random.randn(dimensions[l], 1)

    return param

def forward_propagation(X, param):
    activations = {'A0' : X}
    L = len(param)//2
    
    for l in range(1, L+1):
        Z = param['W' + str(l)].dot(activations['A' + str(l-1)]) + param['b' + str(l)]
        activations['A' + str(l)] = 1 / (1 + np.exp(-Z))
        
    return activations

def back_propagation(Y, activations, param):
    m = Y.shape[1]
    L = len(param) // 2
    
    dZ = activations['A' + str(L)] - Y   
    gradients = {} 
    
    for l in reversed(range(1, L+1)):
        gradients['dW' + str(l)] = 1/m * np.dot(dZ, activations['A' + str(l - 1)].T)
        gradients['db' + str(l)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
        if l > 1:
            dZ = np.dot(param['W' + str(l)].T, dZ) * activations['A' + str(l-1)] * (1-activations['A' + str(l-1)])
    
    return gradients

def update(gradients, param, learning_rate):
    
    L = len(param) // 2
    for l in range(1, L+1):
        param['W' + str(l)] = param['W' + str(l)] - learning_rate * gradients['dW' + str(l)]
        param['b' + str(l)] = param['b' + str(l)] - learning_rate * gradients['db' + str(l)]
    
    return param

def predict(X, param):
    activations = forward_propagation(X, param)
    L = len(param) // 2
    Af = activations['A'+str(L)]
    return (Af >= 0.5)

def normalisedAndReshape(set):    
    return set.reshape(set.shape[0], -1)/set.max()

def affichage_resultat(lossAcc):
    plt.figure(figsize=(16, 9))
    plt.subplot(2, 1, 1)
    plt.plot(lossAcc[0], label='train loss')
    plt.legend()
    plt.subplot(2, 1, 2)
    plt.plot(lossAcc[1], label='train acc')
    plt.legend()
    plt.show()

def log_loss(Y, A):
    e = 1e-15
    return 1/len(Y) * np.sum(-Y * np.log(A + e) - (1 - Y) * np.log(1 - A + e))
            
def load_data():
    train_dataset = h5py.File('datasets/trainset.hdf5', "r")
    X_train = np.array(train_dataset["X_train"][:]) # your train set features
    y_train = np.array(train_dataset["Y_train"][:]) # your train set labels

    test_dataset = h5py.File('datasets/testset.hdf5', "r")
    X_test = np.array(test_dataset["X_test"][:]) # your train set features
    y_test = np.array(test_dataset["Y_test"][:]) # your train set labels
    
    return X_train, y_train, X_test, y_test 

                     
from sklearn.metrics import accuracy_score


In [5]:
#Fonction principale de train avec comparaison du test
def neural_network(X, Y, hidden_layers = (2, 4, 8), learning_rate=0.1, n_iter = 1000, nb_affichage=1000):
    # Initialisation des variables : --------------------------------------------------------------
    dimensions = list(hidden_layers)
    dimensions.insert(0, X.shape[0])
    dimensions.append(Y.shape[0])
    print(dimensions)
    loss_acc = ([],[])
    history_of = []
    # gestion de l'affichage : --------------------------------------------------------------------
    if n_iter <= nb_affichage:
        x_sample = 1
    else:
        x_sample = int((n_iter/nb_affichage))
    
    
    # Initialisation du trainning : ---------------------------------------------------------------
    np.random.seed(0)
    param = initialisation(dimensions) # initialisation des poids du modèle et du biais

    # Aprendtissage : -----------------------------------------------------------------------------
    for i in tqdm(range(n_iter)):
        activations = forward_propagation(X, param)
        gradients = back_propagation(Y, activations, param)
        param = update(gradients, param, learning_rate)
        
        if i % x_sample == 0 :
            L = len(param) // 2
            y_pred = predict(X, param)
            loss_acc[0].append(log_loss(Y, activations['A'+str(L)]))
            loss_acc[1].append(accuracy_score(Y.flatten(), y_pred.flatten()))       
        

    # Affichage des résultats : -------------------------------------------------------------------
    affichage_resultat(loss_acc)
    
    return param

In [6]:
#Récupération des données :
X_train, y_train, X_test, y_test = load_data()

# Normalisation et flatten des données :
normFlat_X_train = normalisedAndReshape(X_train)
normFlat_X_test = normalisedAndReshape(X_test)

normFlat_X_train = normFlat_X_train.T
normFlat_X_test = normFlat_X_test.T
y_train = y_train.T
y_test = y_test.T

print("\n", normFlat_X_train.shape, y_train.shape, normFlat_X_test.shape, y_test.shape, "\n")


 (4096, 1000) (1, 1000) (4096, 200) (1, 200) 



In [7]:
# lancement de l'apprentissage
param = neural_network(normFlat_X_train, y_train, hidden_layers = (64, 64, 64), learning_rate=0.1, n_iter = 4096, nb_affichage=1000)

[4096, 64, 64, 64, 1]


  7%|▋         | 268/4096 [00:06<01:33, 41.14it/s]


KeyboardInterrupt: 

In [175]:
def test_neural_network(X, Y, param, name_set):
    # Aprendtissage : -----------------------------------------------------------------------------
    y_pred = predict(X, param)
    acc = (accuracy_score(Y.flatten(), y_pred.flatten()))       
    print(f"Accuracy score of {acc} with {name_set}")
    
test_neural_network(normFlat_X_test, y_test, param, "Test set")
test_neural_network(normFlat_X_train, y_train, param, "Training set")

Accuracy score of 0.54 with Test set
Accuracy score of 0.981 with Training set


Accuracy score of 0.555 with Test set
Accuracy score of 1.0 with Training set